In [1]:
import pandas as pd
import numpy as np

In [2]:
academic_cycle = pd.read_excel("C:/Users/112363/OneDrive - Taylor's Education Group/DWH_WIP/ImportDateStartNEndDate.xlsx")

In [5]:
academic_cycle.head(2)

,IntakeYear,Cycle,StartDate,EndDate
0,2020,C1,2019-11-01,2020-05-08
1,2021,C1,2020-10-30,2021-04-02


In [4]:
academic_cycle = academic_cycle.drop(['Unnamed: 5', 'Unnamed: 6', 'Email', 'Email Sent', 'No'], axis=1)

In [6]:
academic_cycle = academic_cycle.rename(columns={'IntakeYear': 'intakeyear','Cycle': 'cycle','StartDate': 'startdate','EndDate': 'enddate'})

In [7]:
academic_cycle.head(2)

,intakeyear,cycle,startdate,enddate
0,2020,C1,2019-11-01,2020-05-08
1,2021,C1,2020-10-30,2021-04-02


In [8]:
from r2r_pipelines import export_db

In [36]:
import sqlalchemy
from sqlalchemy import types as sql_types

In [37]:
engine = export_db.marcommdb_connection()
academic_cycle.to_sql('dim_academic_cycle', engine, schema='staging', if_exists='replace', index=False,dtype={
        'startdate': sqlalchemy.types.Date(),
        'enddate': sqlalchemy.types.Date(),    
        'cycle': sqlalchemy.types.String(),
        'intakeyear': sqlalchemy.types.Integer()}
    )

24

In [23]:
academic_cycle_week = academic_cycle.copy()

In [24]:
academic_cycle_week

,intakeyear,cycle,startdate,enddate
0,2020,C1,2019-11-01,2020-05-08
1,2021,C1,2020-10-30,2021-04-02
2,2022,C1,2021-10-29,2022-04-01
3,2020,C2,2020-01-24,2020-07-31
4,2021,C2,2021-01-22,2021-07-30
5,2022,C2,2022-01-21,2022-07-29
6,2020,C3,2020-05-22,2021-01-01
7,2021,C3,2021-05-21,2021-12-31
8,2022,C3,2022-05-20,2022-12-30
9,2023,C1,2022-10-28,2023-03-31


In [25]:
academic_cycle_week['startdate'] = pd.to_datetime(academic_cycle_week['startdate'])
academic_cycle_week['enddate'] = pd.to_datetime(academic_cycle_week['enddate'])

In [26]:
# List to store the weekly breakdown
cycle_week_rows = []

# Loop through each row to generate weekly cycle dates and numbers
for _, row in academic_cycle_week.iterrows():
    year = row['intakeyear'] if 'intakeyear' in row else row.get('year')
    cycle = row['cycle']
    start = row['startdate']
    end = row['enddate']
    
    # Generate weekly dates between StartDate and EndDate (weeks start Monday)
    weekly_dates = pd.date_range(start=start, end=end, freq='W-FRI')

    for week_no, week_date in enumerate(weekly_dates, start=1):
        cycle_week_rows.append({
            'year': year,
            'cycle': cycle,
            'cycle_week_date': week_date,
            'cycle_week_no': week_no
        })


In [27]:
cycle_week_df = pd.DataFrame(cycle_week_rows)

In [28]:
cycle_week_df

,year,cycle,cycle_week_date,cycle_week_no
0,2020,C1,2019-11-01,1
1,2020,C1,2019-11-08,2
2,2020,C1,2019-11-15,3
3,2020,C1,2019-11-22,4
4,2020,C1,2019-11-29,5
...,...,...,...,...
670,2025,C3,2025-11-28,29
671,2025,C3,2025-12-05,30
672,2025,C3,2025-12-12,31
673,2025,C3,2025-12-19,32


In [34]:
cycle_week_df

,intakeyear,cycle,cycle_week_date,cycle_week_no
0,2020,C1,2019-11-01,1
1,2020,C1,2019-11-08,2
2,2020,C1,2019-11-15,3
3,2020,C1,2019-11-22,4
4,2020,C1,2019-11-29,5
...,...,...,...,...
670,2025,C3,2025-11-28,29
671,2025,C3,2025-12-05,30
672,2025,C3,2025-12-12,31
673,2025,C3,2025-12-19,32


In [33]:
cycle_week_df = cycle_week_df.rename(columns={'year': 'intakeyear'})

In [32]:
import sqlalchemy
from sqlalchemy import types as sql_types

In [ ]:
engine = export_db.marcommdb_connection()
cycle_week_df.to_sql('academic_cycle_week', engine, schema='public', if_exists='replace', index=False,dtype={
        'cycle_week_date': sqlalchemy.types.Date(),  
        'cycle': sqlalchemy.types.String(),
        'cycle_week_no': sqlalchemy.types.Integer(),
        'intakeyear': sqlalchemy.types.Integer()
    })

675